In [8]:
from mmcv import Config
from mmseg.apis.inference import init_segmentor
import torch

In [9]:
cfg = Config.fromfile("configs/mic/GTAtoCityscapes.py")

cfg.model.pretrained = None

In [10]:
cfg.load_from = "work_dirs/local-basic/250416_1343_GTAtoCityscapes_22941/latest.pth"

In [11]:
model = init_segmentor(cfg)

/home/sow/Documents/domain_adaptation/PRISM-UDA/mmseg/models/backbones/mix_transformer.py:216: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '


In [14]:
print(type(model))

<class 'mmseg.models.segmentors.hrda_encoder_decoder.HRDAEncoderDecoder'>
